In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



In [4]:
link_list = pd.read_csv('links_with_sitemap.csv')
link_list

,Unnamed: 0,links,status_code,filtered_text,base_link,sitemap_link,sitemap_status
0,0,https://www.factorybuys.com.au/products/euro-t...,200,"['Factory Buys 32cm Euro Top Mattress - King',...",https://www.factorybuys.com.au,https://www.factorybuys.com.au/sitemap.xml,200
1,1,https://dunlin.com.au/products/beadlight-cirrus,200,['Beadlight Cirrus LED Reading Light\n– Dunlin...,https://dunlin.com.au,https://dunlin.com.au/sitemap.xml,200
2,2,https://themodern.net.au/products/hamar-plant-...,200,"['Hamar Plant Stand - Ash\n– The Modern', 'Sof...",https://themodern.net.au,https://themodern.net.au/sitemap.xml,200
3,3,https://furniturefetish.com.au/products/oslo-o...,200,[],https://furniturefetish.com.au,https://furniturefetish.com.au/sitemap.xml,200
4,6,https://interiorsonline.com.au/products/interi...,200,"['Sofas', 'Sofas', '2 Seater Sofas', '2 Seater...",https://interiorsonline.com.au,https://interiorsonline.com.au/sitemap.xml,200
...,...,...,...,...,...,...,...
285,692,https://odafurniture.com/products/,200,[],https://odafurniture.com,https://odafurniture.com/sitemap.xml,200
286,694,https://squarehome.com/products/,200,"['Each range has its own DNA, but all designs ...",https://squarehome.com,https://squarehome.com/sitemap.xml,200
287,696,https://thebeachfurniture.com.au/products/gift...,200,"['SOFAS & LOUNGES', 'ARMCHAIRS', 'COFFEE TABLE...",https://thebeachfurniture.com.au,https://thebeachfurniture.com.au/sitemap.xml,200
288,700,http://aonefurniture.in/userdata/products/no%2...,200,[],http://aonefurniture.in,http://aonefurniture.in/sitemap.xml,200


In [5]:

def check_status(x):
  try:
    res = requests.get(x)
    return res.status_code
  except Exception:
    return 400
link_list['status_code'] = link_list.links.apply(lambda x: check_status(x))

In [4]:
link_list.drop(link_list[link_list.status_code!=200].index, inplace=True)

In [5]:
link_list.to_csv('working_links.csv')

In [11]:
regex_pattern = r'^.+?[^\/:](?=[?\/]|$)'

In [17]:
import re

pattern = re.compile(r'^.+?[^\/:](?=[?\/]|$)')
def get_base_link(link):
    pattern_match = re.match(pattern,link)
    return pattern_match.group(0)
link_list['base_link'] = link_list.links.apply(lambda x: get_base_link(x))
link_list.head()

link_list['sitemap_link'] = link_list.base_link+'/sitemap.xml'

In [19]:
link_list['sitemap_status'] = link_list.sitemap_link.apply(lambda x: check_status(x))

In [20]:
link_list.sitemap_status.value_counts()

sitemap_status
200    269
404     19
500      1
403      1
Name: count, dtype: int64

In [27]:
link_list.to_csv('links_with_sitemap.csv')

In [6]:
link_list[link_list.sitemap_status==200][['sitemap_link']]

,sitemap_link
0,https://www.factorybuys.com.au/sitemap.xml
1,https://dunlin.com.au/sitemap.xml
2,https://themodern.net.au/sitemap.xml
3,https://furniturefetish.com.au/sitemap.xml
4,https://interiorsonline.com.au/sitemap.xml
...,...
284,https://www.collectioni.com/sitemap.xml
285,https://odafurniture.com/sitemap.xml
286,https://squarehome.com/sitemap.xml
287,https://thebeachfurniture.com.au/sitemap.xml


In [64]:
def get_products_name_from_sitemap(sitemap_link):
    res = requests.get(sitemap_link)
    soup = BeautifulSoup(res.text, features='xml')
    product_list = []

    for link in soup.findAll('loc'):

        if 'sitemap_products' in link.text:

            sitemap_res = requests.get(link.text)
            link_soup = BeautifulSoup(sitemap_res.text, features='xml')

            product_list.extend([product.text for product in link_soup.findAll('image:title')])

    return product_list

In [77]:
def get_products_name_link_from_sitemap(sitemap_link):
    res = requests.get(sitemap_link)
    soup = BeautifulSoup(res.text, features='xml')
    product_list = []
    product_link_list = []
    for link in soup.findAll('loc'):

        if 'sitemap_products' in link.text:

            sitemap_res = requests.get(link.text)
            link_soup = BeautifulSoup(sitemap_res.text, features='xml')

            product_list.extend([product.text for product in link_soup.findAll('image:title')][:16:4])
            product_link_list.extend([link.text for link in link_soup.findAll('loc')[1:] if 'shopify' not in link.text][:16:4])

    return list(zip(product_list,product_link_list))

In [93]:

link_list['products_with_links'] = link_list.sitemap_link.apply(lambda x: get_products_name_link_from_sitemap(x))
link_list['product_amount'] = link_list.products.apply(lambda x: len(x))
data_for_training = link_list[link_list.product_amount>0].copy()

In [96]:
all_products_with_links = [product_set for product_list in data_for_training.products_with_links.values for product_set in product_list]
all_products_with_links[:2]

[('Set of 2 Liosia PU Leather Patterned Bar Stools - White & Chrome',
  'https://www.factorybuys.com.au/products/2-x-premium-white-leather-bar-stools'),
 ('Fire Pit BBQ Grill Smoker Table Outdoor Garden Ice Pits Wood Firepit',
  'https://www.factorybuys.com.au/products/outdoor-fire-pit-bbq-table-grill-fireplace-ice-bucket-w-table-lid-free-shipping'),
 ('Kids Ride On Motorbike Motorcycle Car Toys - White',
  'https://www.factorybuys.com.au/products/kids-ride-on-motorbike-white-free-shipping'),
 ('Dog Cage 42inch Pet Cage - Black',
  'https://www.factorybuys.com.au/products/42-inch-metal-collapsible-dog-cage-free-shipping'),
 ('Set of 4 Livorno Bar Stools Gas lift Swivel Steel Chocolate - Steel & Choco',
  'https://www.factorybuys.com.au/products/artiss-set-of-4-bar-stools-kitchen-swivel-bar-stool-pu-leather-gas-lift-chairs-white'),
 ('Mystique Trundle Wooden Bed Frame Timber Storage Drawers Base - Single',
  'https://www.factorybuys.com.au/products/artiss-queen-size-wooden-bed-frame-kid

In [88]:
def get_product_description(link):
    res=requests.get(link)
    soup = BeautifulSoup(res.text)
    blocklist = [
  'style',
  'script',
  # other elements,
]

    text_elements = [t.strip() for t in soup.find_all(string=True) if t.parent.name not in blocklist and len(t.strip())>0]
    text_elements = [t for t in text_elements if t[0] not in ['[','{','<'] and len(t.split())>15]
    return text_elements[1:]


In [117]:
def get_product_description_list(num, product, link):
    try:
        description = get_product_description(link)[0]
    except IndexError:
        description=None
    ner_list = []
    for (i,word) in enumerate(product.split()):
        if i==0:
            ner_list.append([num, word, 'B-PRODUCT'])
        else:
            ner_list.append([num, word, 'I-PRODUCT'])
    if description:
        for word in description.split():
            ner_list.append([num, word, 'O'])
    return ner_list

In [98]:
from tqdm import tqdm

In [104]:
ner_data_list = []
for (num,(name,link)) in enumerate(tqdm(all_products_with_links)):
    ner_data_list.extend(get_product_description_list(num, name,link))

100%|██████████| 1134/1134 [08:40<00:00,  2.18it/s]


In [107]:
ner_df = pd.DataFrame(ner_data_list, columns=['sentence', 'word', 'tag'])

In [108]:
ner_df.tag.value_counts()

tag
0            35375
I-PRODUCT     5772
B-PRODUCT     1134
Name: count, dtype: int64

In [109]:
ner_df.to_csv('product_ner_dataset.csv')

In [119]:
ner_df['sentence'] = ner_df.groupby(['sentence'])['word'].transform(lambda x: ' '.join(x))
ner_df['word_labels'] = ner_df.groupby(['sentence'])['tag'].transform(lambda x: ','.join(x))
ner_df.head()

,sentence,word,tag,word_labels
0,Set of 2 Liosia PU Leather Patterned Bar Stool...,Set,B-PRODUCT,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
1,Set of 2 Liosia PU Leather Patterned Bar Stool...,of,I-PRODUCT,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
2,Set of 2 Liosia PU Leather Patterned Bar Stool...,2,I-PRODUCT,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
3,Set of 2 Liosia PU Leather Patterned Bar Stool...,Liosia,I-PRODUCT,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
4,Set of 2 Liosia PU Leather Patterned Bar Stool...,PU,I-PRODUCT,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."


In [120]:
ner_df = ner_df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
ner_df.head()

,sentence,word_labels
0,Set of 2 Liosia PU Leather Patterned Bar Stool...,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
1,Fire Pit BBQ Grill Smoker Table Outdoor Garden...,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
2,Kids Ride On Motorbike Motorcycle Car Toys - W...,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
3,Dog Cage 42inch Pet Cage - Black Featuring a f...,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."
4,Set of 4 Livorno Bar Stools Gas lift Swivel St...,"B-PRODUCT,I-PRODUCT,I-PRODUCT,I-PRODUCT,I-PROD..."


In [125]:
ner_df.to_csv('ner_dataset_ready_to_train.csv')